In [1]:
import os
import pandas as pd
import pandera as pa
import seaborn as sns
%cd ..
from src.data.dataset import load_kaggle_data

d:\Projects\my-ds-template


This notebook shows how to prepare a tidy dataset from raw data to a state which is ready for further analysis/modeling.  
It includes:  
1. Loading 3 tables with information about the characters from the video game Genshin Impact.
2. What is a tidy dataset and how to deal with common problems in messy data.
3. Cleaning data, handle missing values, update incorrect values.
4. Using DataFrameSchema from Pandera to validate data.


![Why&how to maintain a tidy dataset?](https://cfss.uchicago.edu/media/allison_horst_art/tidydata_1_hua74af901d7485168e9b2819a5f7ffe89_1079942_1200x1200_fit_q90_lanczos.jpg)



## 1. Load all related data
This section covers following operations:
- Retrieving (a subset of) rows
- Sorting
- Testing
- Create/Transform columns
- pivot/melt

### 1.1 Messy data  

3 tables are loaded, from which the common problems of messy data are to be identified:
1. Column headers are values, not variable names.  
   - Solution: melting
2. Multiple variables are stored in one column.  
   - Solution: Separate information into more columns.
3. Variables are stored in both rows and columns.  
   - Solution: melt, pivot  
4. Multiple types of observational units are stored in the same table. 
   - Solution: Separate into different tables.
5. A single observational unit is stored in multiple tables.
   - Solution: concatenate, merge(join)

### 1.2 Load and Observe

In [2]:
dataset_path = 'https://www.kaggle.com/datasets/genshinplayer/genshin-impact-characters-stats'
path, files = load_kaggle_data(dataset_path)
df1 = pd.read_csv(os.path.join(path,files[0])) # skiprows, nrows, header
df1[df1.Character=='Amber']

Skipping, found downloaded files in "../data/raw/genshin-impact-characters-stats" (use force=True to force download)


,Character,Lv,Rarity,Element,Weapon,Main role,Ascension,Base HP,Base ATK,Base DEF
0,Amber,1,4,Pyro,Bow,Sub DPS,ATK,793,19,50
1,Amber,20,4,Pyro,Bow,Sub DPS,ATK,2038,48,129
2,Amber,20,4,Pyro,Bow,Sub DPS,ATK,2630,62,167
3,Amber,40,4,Pyro,Bow,Sub DPS,ATK,3940,93,250
4,Amber,40,4,Pyro,Bow,Sub DPS,ATK,4361,103,277
5,Amber,50,4,Pyro,Bow,Sub DPS,ATK,5016,118,318
6,Amber,50,4,Pyro,Bow,Sub DPS,ATK,5578,131,354
7,Amber,60,4,Pyro,Bow,Sub DPS,ATK,6233,147,396
8,Amber,60,4,Pyro,Bow,Sub DPS,ATK,6654,157,422
9,Amber,70,4,Pyro,Bow,Sub DPS,ATK,7309,172,464



Table `df1` is not following 'each variable forms a column'.  

From Lv20 to Lv80 of the same character, there are two different rows for HP, ATK and DEF.
It can be assumed that we are missing a variable here "before/after ascension".  
("Domain knowledge": In the Game, characters have higher base attributes after Ascension at specific levels.)

In [3]:
# Test Character has identical levels
from pandas import testing as tm
for char, c_df in df1.groupby('Character'):
    # Verify that Lv column for each Character is the same
    tm.assert_series_equal(c_df['Lv'], df1.head(14)['Lv'], check_index=False)
    # Verify that the order of the rows is always before Ascension -> after Ascension
    assert c_df['Base HP'].is_monotonic_increasing==True

No AssertionError, assumption verified.   
If the rows were not in order, .sort_values(by=column_name) can be used to reorder. 

In [4]:
# Sample all rows from df1 will disrupt the order
# .sort_values method supports multiple columns
# it will sort values according to the first column
# if there are identical values, then refer to the next column.
df1.sample(len(df1)).sort_values(by=['Base ATK','Character']).reset_index(drop=True)  

,Character,Lv,Rarity,Element,Weapon,Main role,Ascension,Base HP,Base ATK,Base DEF
0,Hutao,1,5,Pyro,Polearm,DPS,CRIT DMG,1211,8,68
1,Barbara,1,4,Hydro,Catalyst,Healer,HP,821,13,56
2,Sucrose,1,4,Anemo,Catalyst,Support,Anemo DMG,775,14,59
3,Bennett,1,4,Pyro,Sword,Healer,Energy Recharge,1039,16,65
4,Noelle,1,4,Geo,Claymore,Support,DEF,1012,16,67
...,...,...,...,...,...,...,...,...,...,...
569,Ganyu,90,5,Cryo,Bow,DPS,CRIT DMG,9797,335,630
570,Shogun,90,5,Electro,Polearm,Sub DPS,Energy Recharge,12907,337,789
571,Ayaka,90,5,Cryo,Sword,DPS,CRIT DMG,12858,342,784
572,Eula,90,5,Cryo,Claymore,DPS,CRIT DMG,13226,342,751


In [5]:
# Rename columns. Good names are important!
df1 = df1.rename({'Ascension':'Ascension Stat'},axis=1)
# Create a new column representing whether it is before or after Ascension
df1['Ascension'] = len(set(df1.Character))*([pd.NA]+[0,1]*6+[pd.NA])
df1.head(5)

,Character,Lv,Rarity,Element,Weapon,Main role,Ascension Stat,Base HP,Base ATK,Base DEF,Ascension
0,Amber,1,4,Pyro,Bow,Sub DPS,ATK,793,19,50,<NA>
1,Amber,20,4,Pyro,Bow,Sub DPS,ATK,2038,48,129,0
2,Amber,20,4,Pyro,Bow,Sub DPS,ATK,2630,62,167,1
3,Amber,40,4,Pyro,Bow,Sub DPS,ATK,3940,93,250,0
4,Amber,40,4,Pyro,Bow,Sub DPS,ATK,4361,103,277,1


In table `df1`, `Lv` is repeating for every character from 1 to 90.  
Columns like `Rarity`, `Element` do not change with `Lv`.  
Columns like `Base HP` do.   

In [6]:
# pivot can be used to reshape long format table to wide format
# Lv should not be duplicated
reshaped = df1.drop_duplicates(['Character', 'Lv']).pivot(index='Character',columns=['Lv'], values=['Base HP'])
# Table reshaped from long format to wide format
reshaped.head(3)

Base HP                                             
Lv             1     20    40    50    60     70     80     90
Character                                                     
Albedo       1030  2671  5317  6839  8579  10119  11669  13226
Aloy          848  2201  4382  5636  7070   8339   9616  10899
Amber         793  2038  3940  5016  6233   7309   8385   9461

![Long vs Wide Format Table](https://www.statology.org/wp-content/uploads/2021/12/wideLong1-1-768x543.png)

In [7]:
# melt can be used to reshape wide format table to long format
reshaped.columns = reshaped.columns.map('{0[0]}|{0[1]}'.format) 
reshaped.reset_index().melt(id_vars=['Character']).sample(5)

,Character,variable,value
183,Kokomi,Base HP|60,8738
208,Ayaka,Base HP|70,9838
314,Sara,Base HP|90,9570
212,Chongyun,Base HP|70,8485
251,Beidou,Base HP|80,11565


In [8]:
# Second data source include additional information on Region and Model Type
url= 'https://genshin-impact.fandom.com/wiki/Characters'
df2 = pd.read_html(url)[2]
df2 = df2.drop(columns=['Icon','Rarity'])
df2 = df2.rename({'Name':'Character'},
                 axis=1)
df2.sample(3)  #  head, tail                                                    

,Character,Element,Weapon,Region,Model Type
13,Gorou,Geo,Bow,Inazuma,Medium Male
16,Kaedehara Kazuha,Anemo,Sword,Inazuma,Medium Male
43,Xingqiu,Hydro,Sword,Liyue,Medium Male


In `df2`, Model Type contains 2 variables: height and gender.

In [9]:
df2['height'] = df2['Model Type'].transform(lambda x: x.split(' ')[0])
df2['gender'] = df2['Model Type'].transform(lambda x: x.split(' ')[1])
df2.drop(columns=['Model Type']).sample(3)

,Character,Element,Weapon,Region,height,gender
43,Xingqiu,Hydro,Sword,Liyue,Medium,Male
1,Aloy,Cryo,Bow,None,Medium,Female
22,Kujou Sara,Electro,Bow,Inazuma,Tall,Female


In [10]:
# Third data source include stats of more Characters at Lv90
url= 'https://genshin-impact.fandom.com/wiki/Characters/Comparison'
df3 = pd.read_html(url)[1]
df3 = df3.drop(columns=['Icon'])
df3.rename(columns={'Name':'Character',
                    'HP': 'Base HP',
                    'ATK': 'Base ATK',
                    'DEF': 'Base DEF',
                    }, 
           inplace=True)
df3.head(4)

,Character,Base HP,Base ATK,Base DEF,Ascension Stat,Ascension Stat Value
0,Albedo,13225,251,876,Geo DMG Bonus,28.8%
1,Aloy,10898,233,676,Cryo DMG Bonus,28.8%
2,Amber,9461,223,600,ATK,24.0%
3,Arataki Itto,12858,227,959,CRIT Rate,19.2%


## 2. Tidy Data

Operations:  

- String processing
- Union, Intersection, Difference, Symmetric Difference of sets
- Merge (Join) multiple tables  
    A [helpful question on Stack Overflow](https://stackoverflow.com/questions/53645882/pandas-merging-101/53645883#53645883).

This section is to create tidy datasets from the 3 data frames loaded in section 1.


### 2.1 Create an ID for each Character
To match the rows from different tables to the corresponding characters.

In [11]:
# Table 1: 
# Only first name for japanese names
df1.Character.unique()

array(['Amber', 'Barbara', 'Beidou', 'Bennett', 'Chongyun', 'Diluc',
       'Fischl', 'Jean', 'Kaeya', 'Keqing', 'Klee', 'Lisa', 'Mona',
       'Ningguang', 'Noelle', 'Qiqi', 'Razor', 'Sucrose', 'Venti',
       'Xiangling', 'Xiao', 'Xingqiu', 'Tartaglia', 'Zhongli', 'Diona',
       'Xinyan', 'Ganyu', 'Albedo', 'Rosaria', 'Ayaka', 'Hutao', 'Yanfei',
       'Eula', 'Kazuha', 'Yoimiya', 'Sayu', 'Traveler', 'Shogun', 'Aloy',
       'Sara', 'Kokomi'], dtype=object)

In [12]:
# Table 2&3 have identical Characters
assert (df3.Character.unique() == df2.Character.unique()).all()
df3.Character.unique()

array(['Albedo', 'Aloy', 'Amber', 'Arataki Itto', 'Barbara', 'Beidou',
       'Bennett', 'Chongyun', 'Diluc', 'Diona', 'Eula', 'Fischl', 'Ganyu',
       'Gorou', 'Hu Tao', 'Jean', 'Kaedehara Kazuha', 'Kaeya',
       'Kamisato Ayaka', 'Kamisato Ayato', 'Keqing', 'Klee', 'Kujou Sara',
       'Kuki Shinobu', 'Lisa', 'Mona', 'Ningguang', 'Noelle', 'Qiqi',
       'Raiden Shogun', 'Razor', 'Rosaria', 'Sangonomiya Kokomi', 'Sayu',
       'Shenhe', 'Shikanoin Heizou', 'Sucrose', 'Tartaglia', 'Thoma',
       'Traveler', 'Venti', 'Xiangling', 'Xiao', 'Xingqiu', 'Xinyan',
       'Yae Miko', 'Yanfei', 'Yelan', 'Yoimiya', 'Yun Jin', 'Zhongli'],
      dtype=object)

We can use the lowercase of the full names without spaces. 

In [13]:
# Create unique IDs for characters based on their full names in table 2
df2['ID'] = df2['Character'].transform(lambda x: x.replace(' ','').lower())
df3['ID'] = df3['Character'].transform(lambda x: x.replace(' ','').lower())

# Assign ID to characters in table 1
def get_id(x, id_list):
    for id in id_list:
        if x.lower() in id:
            return id
    return x
df1['ID'] = df1['Character'].apply(lambda x: get_id(x, df2['ID'].unique()))

In [14]:
# Intersection, Difference, Union of sets
len(set(df2.ID)|set(df1.ID)) # &, -, | 

51

In [15]:
# All ID in df1 can be found in df2
set(df1.ID) - set(df2.ID)

set()

In [16]:
set(df2.columns) & set(df1.columns)

{'Character', 'Element', 'ID', 'Weapon'}

In [17]:
df1

,Character,Lv,Rarity,Element,Weapon,Main role,Ascension Stat,Base HP,Base ATK,Base DEF,Ascension,ID
0,Amber,1,4,Pyro,Bow,Sub DPS,ATK,793,19,50,<NA>,amber
1,Amber,20,4,Pyro,Bow,Sub DPS,ATK,2038,48,129,0,amber
2,Amber,20,4,Pyro,Bow,Sub DPS,ATK,2630,62,167,1,amber
3,Amber,40,4,Pyro,Bow,Sub DPS,ATK,3940,93,250,0,amber
4,Amber,40,4,Pyro,Bow,Sub DPS,ATK,4361,103,277,1,amber
...,...,...,...,...,...,...,...,...,...,...,...,...
569,Kokomi,70,5,Hydro,Catalyst,Healer,Hydro DMG,10306,179,503,0,sangonomiyakokomi
570,Kokomi,70,5,Hydro,Catalyst,Healer,Hydro DMG,10945,190,534,1,sangonomiyakokomi
571,Kokomi,80,5,Hydro,Catalyst,Healer,Hydro DMG,11885,207,580,0,sangonomiyakokomi
572,Kokomi,80,5,Hydro,Catalyst,Healer,Hydro DMG,12524,218,611,1,sangonomiyakokomi


### 2.2 Separate different types of observation
As mentioned in 1.1 and 1.2, a table is suggested to have only one type of observation.
In the following cell, df1 is broken down into a character dataset with their constant characteristics, and a character growth dataset with their attributes changes from Lv1 to Lv 90.  

In [18]:
df_growth = df1[['ID', 'Character', 'Lv', 'Ascension', 'Base HP', 'Base ATK', 'Base DEF']]
df_growth.head(3)

,ID,Character,Lv,Ascension,Base HP,Base ATK,Base DEF
0,amber,Amber,1,<NA>,793,19,50
1,amber,Amber,20,0,2038,48,129
2,amber,Amber,20,1,2630,62,167


In [19]:
df_character = df1[['ID', 'Rarity', 'Main role']].drop_duplicates().reset_index(drop=True)
df_character.head(3)

,ID,Rarity,Main role
0,amber,4,Sub DPS
1,barbara,4,Healer
2,beidou,4,Sub DPS


In [20]:
df2['ID'] = df2['Character'].transform(lambda x: x.replace(' ','').lower())
columns_from_df2 = ['ID']+list(set(df2.columns) - set(df1.columns))
df_character = pd.merge(df2[columns_from_df2], df_character, on='ID', how='left')
df_character = df_character.set_index('ID').reset_index()
df_character.head(3)

,ID,height,Region,Model Type,gender,Rarity,Main role
0,albedo,Medium,Mondstadt,Medium Male,Male,5.0,Support
1,aloy,Medium,None,Medium Female,Female,5.0,DPS
2,amber,Medium,Mondstadt,Medium Female,Female,4.0,Sub DPS


In [21]:
columns_from_df3 = ['ID']+list(set(df3.columns) - set(df_character.columns))
df_character = pd.merge(df_character, df3[columns_from_df3],  on='ID', how='left')
df_character = df_character.set_index('ID').reset_index()
df_character.sample(3)

,ID,height,Region,Model Type,gender,Rarity,Main role,Ascension Stat Value,Base DEF,Base HP,Character,Ascension Stat,Base ATK
5,beidou,Tall,Liyue,Tall Female,Female,4.0,Sub DPS,24.0%,648,13049,Beidou,Electro DMG Bonus,225
16,kaedeharakazuha,Medium,Inazuma,Medium Male,Male,5.0,Support,115.2,806,13348,Kaedehara Kazuha,Elemental Mastery,296
46,yanfei,Medium,Liyue,Medium Female,Female,4.0,DPS,24.0%,586,9352,Yanfei,Pyro DMG Bonus,240


## 3. Clean data
- Check & fill missing values (na/null)  
- Update cells according to index and label


In [22]:
# Find mismatched values in Lv 90 stats
columns = ['ID','Base HP','Base ATK','Base DEF']

hp_df1 = df_character[columns].copy()
hp_df1['tabel'] = 'character'

hp_df3 = df_growth[df_growth.Lv==90][columns].copy()
hp_df3['tabel'] = 'growth'

pd.concat([hp_df1, hp_df3], axis=0).drop_duplicates(subset=columns, keep=False).sort_values('ID')

,ID,Base HP,Base ATK,Base DEF,tabel
0,albedo,13225,251,876,character
391,albedo,13226,251,876,growth
1,aloy,10898,233,676,character
545,aloy,10899,234,676,growth
2,amber,9461,223,600,character
...,...,...,...,...,...
48,yoimiya,10164,322,614,character
489,yoimiya,10164,323,615,growth
49,yunjin,10657,191,734,character
50,zhongli,14695,251,737,character


In [23]:
df_growth.isna().sum()

ID            0
Character     0
Lv            0
Ascension    82
Base HP       0
Base ATK      0
Base DEF      0
dtype: int64

In [24]:
df_character.isna().sum()

ID                       0
height                   0
Region                   0
Model Type               0
gender                   0
Rarity                  10
Main role               10
Ascension Stat Value     0
Base DEF                 0
Base HP                  0
Character                0
Ascension Stat           0
Base ATK                 0
dtype: int64

In [25]:
df_character[df_character.Character.isna()]

,ID,height,Region,Model Type,gender,Rarity,Main role,Ascension Stat Value,Base DEF,Base HP,Character,Ascension Stat,Base ATK


In [26]:
# Sometimes, if we have to, we can update values manually.
# Difference between loc & iloc see https://stackoverflow.com/questions/31593201/how-are-iloc-and-loc-different
df_character.loc[[1,39], ['Region']] = [['Nora'], ['Unknown']]
df_character.iloc[[1,39]]

,ID,height,Region,Model Type,gender,Rarity,Main role,Ascension Stat Value,Base DEF,Base HP,Character,Ascension Stat,Base ATK
1,aloy,Medium,Nora,Medium Female,Female,5.0,DPS,28.8%,676,10898,Aloy,Cryo DMG Bonus,233
39,traveler,Aether:,Unknown,Aether: Medium MaleLumine: Medium Female,Medium,5.0,Support,24.0%,682,10874,Traveler,ATK,212


## 4. Data Validation


In [44]:
schema_character = pa.DataFrameSchema(columns={
                                                'ID':  pa.Column(str, nullable=False),
                                                'gender': pa.Column(str, checks=pa.Check.isin(['Male', 'Female']), nullable=True)
                                                },
                                      unique=['ID'],
                                      )

schema_growth = pa.DataFrameSchema(columns={
                                            'ID':  pa.Column(str, nullable=False),
                                            'Lv':   pa.Column(int, checks=pa.Check.isin([1, 20, 40, 50, 60, 70, 80, 90])),
                                            'Base HP':   pa.Column(float, coerce=True),
                                            }
                                   )

In [ ]:
schema_character(df_character)

A schemaError is thrown as the Model Type of the traveler is special.

In [31]:
df_character.loc[df_character.ID=='traveler']

,ID,height,Region,Model Type,gender,Rarity,Main role,Ascension Stat Value,Base DEF,Base HP,Character,Ascension Stat,Base ATK
39,traveler,Aether:,Unknown,Aether: Medium MaleLumine: Medium Female,Medium,5.0,Support,24.0%,682,10874,Traveler,ATK,212


In [32]:
# Fill traveler's gender and height manually
df_character.loc[df_character.ID=='traveler', 'gender'] = pd.NA
df_character.loc[df_character.ID=='traveler', 'height'] = 'Medium'

In [48]:
schema_character(df_character)
schema_growth(df_growth).sample(1)

,ID,Character,Lv,Ascension,Base HP,Base ATK,Base DEF
182,ningguang,Ningguang,1,<NA>,821.0,18,48


In [ ]:
df_growth.to_csv('data/processed/genshin_character_growth.csv', index=False)
df_character.drop(columns=['Model Type']).to_csv('data/processed/genshin_character.csv', index=False)

## Checklist
- Assumptions  
  1. Names in df1 are substrings from df2 after removing spaces and lowercaseization.
- Missing values  
  - Traveler does not have gender.  
  - 
  - Lv1 and Lv90 rows don't have Ascension label.


In [49]:
pd.concat([df_character.head(3),df_character.tail(3)])

,ID,height,Region,Model Type,gender,Rarity,Main role,Ascension Stat Value,Base DEF,Base HP,Character,Ascension Stat,Base ATK
0,albedo,Medium,Mondstadt,Medium Male,Male,5.0,Support,28.8%,876,13225,Albedo,Geo DMG Bonus,251
1,aloy,Medium,Nora,Medium Female,Female,5.0,DPS,28.8%,676,10898,Aloy,Cryo DMG Bonus,233
2,amber,Medium,Mondstadt,Medium Female,Female,4.0,Sub DPS,24.0%,600,9461,Amber,ATK,223
48,yoimiya,Medium,Inazuma,Medium Female,Female,5.0,DPS,19.2%,614,10164,Yoimiya,CRIT Rate,322
49,yunjin,Medium,Liyue,Medium Female,Female,NaN,NaN,26.8%,734,10657,Yun Jin,Energy Recharge,191
50,zhongli,Tall,Liyue,Tall Male,Male,5.0,Support,28.8%,737,14695,Zhongli,Geo DMG Bonus,251


In [50]:
pd.concat([df_growth.head(3),df_growth.tail(3)])

,ID,Character,Lv,Ascension,Base HP,Base ATK,Base DEF
0,amber,Amber,1,<NA>,793,19,50
1,amber,Amber,20,0,2038,48,129
2,amber,Amber,20,1,2630,62,167
571,sangonomiyakokomi,Kokomi,80,0,11885,207,580
572,sangonomiyakokomi,Kokomi,80,1,12524,218,611
573,sangonomiyakokomi,Kokomi,90,<NA>,13471,234,657


In [51]:
# query detailed information for each character on fandom.com
url= 'https://genshin-impact.fandom.com/wiki/Razor'
df = pd.read_html(url)[2]
df.head(3)

,AscensionPhase,Level,BaseHP,BaseATK[1],BaseDEF,Special Stat[2](Physical DMG Bonus)
0,0✦,1/20,1002,19,62,—
1,0✦,20/20,2576,50,161,—
2,"Ascension Cost (0 → 1)20,000 20,000 Mora1 1 Va...","Ascension Cost (0 → 1)20,000 20,000 Mora1 1 Va...","Ascension Cost (0 → 1)20,000 20,000 Mora1 1 Va...","Ascension Cost (0 → 1)20,000 20,000 Mora1 1 Va...","Ascension Cost (0 → 1)20,000 20,000 Mora1 1 Va...","Ascension Cost (0 → 1)20,000 20,000 Mora1 1 Va..."
